In [60]:
from pandas import DataFrame
import pandas as pd
import git
import xlrd
import matplotlib.pyplot as plt
import random
from scipy.stats import wilcoxon
from sqlalchemy import create_engine, or_, Column, Integer, String, Float, DateTime, ForeignKey, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from do_analysis import Experiment, ExperimentSet

In [61]:
engine = create_engine('sqlite:///results.db')
session_class = sessionmaker(bind=engine)
session = session_class()

# Load data into the dataframe

In [105]:
last = ""
headers = []
datasets = []
series = []
serie = None
repo = git.Repo(search_parent_directories=True)

for experiment in session.query(Experiment).order_by(Experiment.file_name):
#for experiment in session.query(Experiment).filter(or_(Experiment.set_id==i for i in [17])).order_by(Experiment.file_name):
    if last != experiment.file_name:
        last = experiment.file_name
        dataset_name = experiment.file_name.rsplit('/')[-1].split(".")[0]
        datasets.append(dataset_name)
        if serie is not None:
            series.append(serie)
        serie = []
    message = repo.commit(experiment.set.commit).message.strip()
    message = experiment.set.description
    measure =  experiment.method.split('.')[-1]
    if "LearningBased" in measure:
        strategy = measure.split(" ")[4]
        weight = measure.split(" ")[6]
        measure = f"LearningBased strategy {strategy}, weight {weight}"
    header = f"{measure} - {message} - {experiment.set.commit}"
    if header not in headers:
        headers.append(header)
    serie.append(experiment.f_score)
series.append(serie)

df = pd.DataFrame(series, index = datasets, columns=headers)
df

,EskinModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,GambaryanModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,GoodallModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,OFModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,IOFModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,LinModified_Kappa - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,LinModified_KappaMax - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,EskinModified - Base measure times (1-kappa) discarding - 425c2e53b4e31b14a4a32ac25bb76f12d90ca62a,GambaryanModified - Base measure times (1-kappa) discarding - 425c2e53b4e31b14a4a32ac25bb76f12d90ca62a,GoodallModified - Base measure times (1-kappa) discarding - 425c2e53b4e31b14a4a32ac25bb76f12d90ca62a,...,"LearningBased strategy C, weight A - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy D, weight N - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy D, weight K - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy D, weight A - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy E, weight N - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy E, weight K - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy E, weight A - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy N, weight N - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy N, weight K - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy N, weight A - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a"
allbp_complete_cleaned,0.956699,0.956699,0.956699,0.956699,0.956699,0.956699,0.956699,0.956699,0.956699,0.956699,...,0.601846,0.829734,0.909016,0.832534,0.815091,0.663440,0.829734,0.956699,0.956699,0.956699
arrhythmia_cleaned,0.487002,0.493951,0.487552,0.486920,0.492585,0.424266,0.488440,0.489008,0.487870,0.487544,...,0.441823,0.427121,0.501966,0.431420,0.425638,0.424684,0.437865,0.488440,0.488440,0.488440
australian_cleaned,0.671404,0.671404,0.671404,0.671404,0.671404,0.674492,0.671404,0.671404,0.671404,0.671404,...,0.727323,0.636628,0.674492,0.727323,0.636628,0.674492,0.727323,0.652566,0.605572,0.727323
cervical_cancer_biopsy_cleaned,0.936098,0.936098,0.936098,0.936098,0.936098,0.936098,0.936098,0.936098,0.936098,0.936098,...,0.926908,0.926908,0.926908,0.926908,0.926908,0.926908,0.926908,0.926908,0.926908,0.926908
cervical_cancer_cytology_cleaned,0.948797,0.948797,0.948797,0.948797,0.948797,0.948797,0.948797,0.948797,0.948797,0.948797,...,0.935389,0.935389,0.935389,0.935389,0.935389,0.935389,0.935389,0.935389,0.935389,0.935389
cervical_cancer_hinselmann_cleaned,0.959229,0.959229,0.959229,0.959229,0.959229,0.959229,0.959229,0.959229,0.959229,0.959229,...,0.945911,0.945911,0.945911,0.945911,0.945911,0.945911,0.945911,0.945911,0.945911,0.945911
cervical_cancer_schiller_cleaned,0.914341,0.914341,0.914341,0.914341,0.914341,0.914341,0.914341,0.914341,0.914341,0.914341,...,0.906946,0.906946,0.906946,0.906946,0.906946,0.906946,0.906946,0.906946,0.906946,0.906946
crx_cleaned,0.671404,0.671404,0.671404,0.671404,0.671404,0.699501,0.671404,0.671404,0.671404,0.671404,...,0.674492,0.674492,0.674492,0.674584,0.674492,0.674492,0.674584,0.671404,0.753200,0.671404
diagnosis1_cleaned,0.564785,0.734113,0.509957,0.564785,0.564785,0.509957,0.509957,0.615792,0.722096,0.722096,...,0.564785,0.567037,0.564785,0.564785,0.567037,0.564785,0.564785,0.567037,0.567037,0.567037
diagnosis2_cleaned,0.744626,0.543498,0.725275,0.744626,0.744626,0.725275,0.725275,0.560440,0.538259,0.538259,...,0.744626,0.643799,0.744626,0.744626,0.643799,0.744626,0.744626,0.643799,

# Rank each value against the other options, per dataset

In [110]:
dft = df.apply(lambda x: x.rank(ascending=False), axis = 1)
dft

,EskinModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,GambaryanModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,GoodallModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,OFModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,IOFModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,LinModified_Kappa - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,LinModified_KappaMax - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,EskinModified - Base measure times (1-kappa) discarding - 425c2e53b4e31b14a4a32ac25bb76f12d90ca62a,GambaryanModified - Base measure times (1-kappa) discarding - 425c2e53b4e31b14a4a32ac25bb76f12d90ca62a,GoodallModified - Base measure times (1-kappa) discarding - 425c2e53b4e31b14a4a32ac25bb76f12d90ca62a,...,"LearningBased strategy C, weight A - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy D, weight N - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy D, weight K - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy D, weight A - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy E, weight N - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy E, weight K - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy E, weight A - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy N, weight N - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy N, weight K - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy N, weight A - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a"
allbp_complete_cleaned,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,...,97.5,60.5,47.0,57.5,63.5,82.0,60.5,23.0,23.0,23.0
arrhythmia_cleaned,45.0,3.0,41.0,47.0,6.0,83.5,22.0,10.5,33.5,42.5,...,49.0,72.5,2.0,65.5,78.5,80.0,51.5,22.0,22.0,22.0
australian_cleaned,65.0,65.0,65.0,65.0,65.0,31.5,65.0,65.0,65.0,65.0,...,7.5,86.5,31.5,7.5,86.5,31.5,7.5,82.5,89.0,7.5
cervical_cancer_biopsy_cleaned,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,...,67.0,67.0,67.0,67.0,67.0,67.0,67.0,67.0,67.0,67.0
cervical_cancer_cytology_cleaned,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,...,67.0,67.0,67.0,67.0,67.0,67.0,67.0,67.0,67.0,67.0
cervical_cancer_hinselmann_cleaned,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,...,68.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0
cervical_cancer_schiller_cleaned,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,...,70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0
crx_cleaned,86.0,86.0,86.0,86.0,86.0,10.5,86.0,86.0,86.0,86.0,...,54.5,54.5,54.5,27.0,54.5,54.5,27.0,86.0,1.5,86.0
diagnosis1_cleaned,80.5,7.5,113.0,80.5,80.5,113.0,113.0,22.0,12.5,12.5,...,80.5,41.0,80.5,80.5,41.0,80.5,80.5,41.0,41.0,41.0
diagnosis2_cleaned,26.5,104.5,55.0,26.5,26.5,55.0,55.0,96.5,109.5,109.5,...,26.5,73.0,26.5,26.5,73.0,26.5,26.5,73.0,73.0,73.0


# Get the mean rank per measure and order them from best to worst

In [107]:
averages = dft.mean()
averages.sort_values()

IOFModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed                                                 23.534483
GambaryanModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed                                           23.603448
IOFModified - Base measures times (1-kappa) - 39dc158f00e49188b8bcb745bbc1ad36ecc7a8c8                                             23.879310
IOFModified - Base measure times (1-kappa) discarding - 425c2e53b4e31b14a4a32ac25bb76f12d90ca62a                                   23.879310
OFModified - Base measure times (1-kappa) discarding - 425c2e53b4e31b14a4a32ac25bb76f12d90ca62a                                    24.086207
OFModified - Base measures times (1-kappa) - 39dc158f00e49188b8bcb745bbc1ad36ecc7a8c8                                              24.086207
EskinModified - Base measures times (1-kappa) - 39dc158f00e49188b8bcb745bbc1ad36ecc7a8c8                                           24.086207
EskinModified

# Compare the best and second best using the Wilcoxon test

In [108]:
best = averages.sort_values().index[0]
second_best = averages.sort_values().index[1]
_, p = wilcoxon(df[best], df[second_best])
p
#for i, a in zip(range(3), averages.sort_values()):
#    print(type(a))

/home/jacob/.venvs/ml/lib/python3.6/site-packages/scipy/stats/morestats.py:2388: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")


1.0

# Check the differences between the best and second best

In [109]:
print(f"{best} | {second_best}")
for i, (x, y) in enumerate(zip(df[best], df[second_best])):
    if x != y: 
        print(f"{df.index[i]}: {x} - {y}")

IOFModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed | GambaryanModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed
arrhythmia_cleaned: 0.492585003376007 - 0.493950545787811
diagnosis1_cleaned: 0.56478488445282 - 0.734113276004791
diagnosis2_cleaned: 0.744625985622406 - 0.543498158454895
